In [112]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [113]:
words=open('names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [114]:
len (words)

32033

In [115]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [152]:
block_size=3
X,Y=[],[]
for w in words:
  #print(w)
  context=[0]* block_size
  for ch in w+'.':
    ix=stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [153]:
X.shape,X.dtype, Y.shape,Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [154]:
C=torch.randn((27,2))

In [155]:
C[5]

tensor([1.0961, 0.5742])

In [156]:
emb=C[X]
emb.shape

torch.Size([228146, 3, 2])

In [157]:
w1=torch.randn((6,100))
b1=torch.randn(100)

In [158]:
h = torch.tanh(emb.view(-1,6)@ w1 + b1)
h.shape

torch.Size([228146, 100])

In [159]:
w2= torch.randn((100,27))
b2=torch.randn(27)

In [160]:
logits= h @ w2 + b2

In [161]:
logits.shape

torch.Size([228146, 27])

In [162]:
counts=logits.exp()

In [163]:
prob = counts / counts.sum(1, keepdims=True)

In [164]:
prob.shape

torch.Size([228146, 27])

In [165]:
prob[0].sum()

tensor(1.0000)

**------------Cleaner Code------------**


In [167]:
X.shape,Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [242]:
g= torch.Generator().manual_seed(2147283647)
C=torch.randn((27,2),generator=g)
W1 = torch.randn((6,100),generator=g)
b1= torch.randn(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2= torch.randn(27,generator=g)
parameters=[C,W1,b1,W2,b2]

In [243]:
sum(p.nelement() for p in parameters)

3481

In [244]:
for p in parameters:
  p.requires_grad = True

In [245]:
print(p.requires_grad)

True


In [246]:
lre = torch.linspace(-3,0,1000)
lrs = 10**lre

Training the entire dataset at once eats up a lot of computation

In [247]:
lri=[]
lossi=[]
for i in range(10000):

  #minibatch construct
  ix = torch.randint(0, X.shape[0],(32,))

  #Forward pass
  emb = C[X[ix]]
  h = torch.tanh(emb.view(-1,6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits,Y[ix])

  #Backward pass
  for p in parameters:
    p.grad = None
  loss.backward()

  #update
  for p in parameters:
    #lr = lrs[i]
    lr=0.1
    p.data += -lr * p.grad

  #track stats
  #lri.append(lre[i])
  #lossi.append(loss.item())

print(loss.item())

2.511542320251465


In [248]:
#plt.plot(lri, lossi)

In [249]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Y)
loss

tensor(2.4596, grad_fn=<NllLossBackward0>)